In [1]:
import hydra
import wandb 
import os
import ddm_stride.__init__
from ddm_stride.pipeline.evaluate import load_experimental_data
import warnings
warnings.filterwarnings('ignore')

# Adapt the path of the results folder
dir = os.environ['DDM_STRIDE_DIR'] + '/results/${result_folder}'

In [ ]:
%run ../ddm_stride/run.py hydra.run.dir={dir} run_simulate=True 

The quickstart tutorial applies a Basic DDM with parameters `drift`, `boundary_separation` and `starting_point` of the decision variable to the [Roitman and Shadlen](https://shadlenlab.columbia.edu/resources/RoitmanDataCode.html) dataset. The dataset contains reaction times and choices of monkeys discriminating between two directions of random dot motion. The pipeline is configured by the *basic_ddm.yaml* configuration files in the *config/algorithm*, *config/ddm_model* and *config/task* folders.

### Simulate training and test data

### Train the MNLE using 2 different hyperparameter configurations
You can enter the user *ddmstride* with password *ddm_stride_22* as a default account for WandB.

In [3]:
wandb.login(relogin=False)

True

In [4]:
%run ../ddm_stride/run.py hydra.run.dir={dir} run_train=True 

[2022-05-30 13:45:19,486][wandb.jupyter][ERROR] - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: ddm_stride. Use `wandb login --relogin` to force relogin


Create sweep with ID: pvjrv9n5
Sweep URL: https://wandb.ai/ddm_stride/basic_ddm/sweeps/pvjrv9n5
[2022-05-30 13:45:21,427][wandb.agents.pyagent][INFO] - Starting sweep agent: entity=None, project=basic_ddm, count=2


wandb: Agent Starting Run: 2ajuuxqx with config:
wandb: 	hidden_features: 50
wandb: 	hidden_layers: 4
wandb: 	learning_rate: 0.0035642783231513873
wandb: 	num_transforms: 5
wandb: 	stop_after_epochs: 20
wandb: 	training_batch_size: 256
wandb: 	validation_fraction: 0.2


[2022-05-30 13:45:21,972][wandb.jupyter][ERROR] - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


 Neural network successfully converged after 43 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 43
        Best validation performance: -0.1943
        -------------------------
        


validation_log_prob,▁
validation_log_prob,-0.19427


wandb: Agent Starting Run: ske2fswl with config:
wandb: 	hidden_features: 25
wandb: 	hidden_layers: 2
wandb: 	learning_rate: 0.000403413057999576
wandb: 	num_transforms: 2
wandb: 	stop_after_epochs: 20
wandb: 	training_batch_size: 256
wandb: 	validation_fraction: 0.2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


 Neural network successfully converged after 127 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 127
        Best validation performance: -0.1781
        -------------------------
        


validation_log_prob,▁
validation_log_prob,-0.17809


### Run the fast diagnosis. 
Pre-computed results of the slow diagnosis can be found in *results/basic_ddm/diagnose*.

In [ ]:
%run ../ddm_stride/run.py hydra.run.dir={dir} run_diagnose_fast=True 

### Run the evaluation on the experimental data
The observations names in *config/ddm_model* correspond to the `rt` and `choice` columns in the experimental data.

In [3]:
with hydra.initialize(config_path='../config'):
    cfg = hydra.compose(config_name='config')

experimental_data = load_experimental_data(cfg)
experimental_data

,monkey,rt,coh,correct,choice
0,1,0.355,0.512,1.0,0.0
1,1,0.359,0.256,1.0,1.0
2,1,0.525,0.128,1.0,1.0
3,1,0.332,0.512,1.0,1.0
4,1,0.302,0.032,0.0,0.0
...,...,...,...,...,...
6144,2,0.627,0.032,1.0,1.0
6145,2,0.581,0.256,1.0,1.0
6146,2,0.293,0.512,1.0,1.0
6147,2,0.373,0.128,1.0,0.0


In [ ]:
%run ../ddm_stride/run.py hydra.run.dir={dir} run_evaluate=True 